**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import tensorflow.keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("/Users/johnlevy/Downloads/ffmpeg-20200129-de1b2aa-macos64-static/bin")
import skvideo.io
import cv2
import json

from tensorflow.keras import backend

from tensorflow.keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

In [3]:
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act gives the agent the next action to do. The epsilon ensures the exploration of the agent, the more the epsilon is large, the more we are going to encourage the exploration, the lower the espilon get, the more the exploitation get. This espilon represents the trade-off between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [35]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2 :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        #on prend aléatoirement de 3 à grid - 3 et [0] pour dire qu'on veut pas un array mais un réel
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0] 
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [36]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array is used to give the reward. The rewards are given by : +0.5 if there is cheese, -1 for poisonous, 0 otherwise. The board represents whee the rat can go : -1 for the board of the grid (these area is forbidden), +1 where the rat stands, 0 otherwise.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [37]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(4)
        return action


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [38]:
def test(agent,env,epochs,prefix=''):
    
    
    # Number of won games
    score = 0
        



    for e in range(epochs):
    # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
    # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
        # Apply an action to the environment, get the next state, the reward
        # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
        if e % 10 == 0:
            env.draw(e)

    # Update stats
        score += win-lose
        
        
        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
        print('Final score: '+str(score/epochs))

In [39]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')


Win/lose count 13.0/15.0. Average score (-4.0)
Final score: -0.2
Win/lose count 9.0/10.0. Average score (-3.0)
Final score: -0.3
Win/lose count 6.0/10.0. Average score (-4.666666666666667)
Final score: -0.7
Win/lose count 10.0/16.0. Average score (-6.5)
Final score: -1.3
Win/lose count 13.5/18.0. Average score (-7.0)
Final score: -1.75
Win/lose count 9.0/18.0. Average score (-8.833333333333334)
Final score: -2.65
Win/lose count 8.5/14.0. Average score (-9.142857142857142)
Final score: -3.2
Win/lose count 9.0/13.0. Average score (-9.0)
Final score: -3.6
Win/lose count 7.5/13.0. Average score (-9.222222222222221)
Final score: -4.15
Win/lose count 11.5/11.0. Average score (-8.2)
Final score: -4.1
Win/lose count 11.5/15.0. Average score (-8.090909090909092)
Final score: -4.45
Win/lose count 9.5/20.0. Average score (-9.166666666666666)
Final score: -5.5
Win/lose count 10.5/11.0. Average score (-8.538461538461538)
Final score: -5.55
Win/lose count 12.5/14.0. Average score (-8.142857142857142

In [40]:
HTML(display_videos('random0.mp4'))

***
## DQN

In [1]:
%%latex

UsageError: %%latex is a cell magic, but the cell body is empty.


Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$\bf{\underline{ First \hspace{0.1cm} question :}} $

$$
Q^{\pi}(s,a) = E_{p^\pi}[\sum_{t\leq T} \gamma^{t} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= E_{p^\pi}[  r(s,a) + \gamma\sum_{t=1}^{T} \gamma^{t-1} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s_1 = s',a_1 = a'|s_0,a_0)E[\sum_{t=1}^{T}\gamma^{t-1}r(s_t,a_t)|s_1 = s', a_1 = a']\\
= r(s,a) + \gamma\sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{\pi}(s',a')\\
=r(s,a) + \gamma E_{s',a'\sim p(.|s,a)}[Q^{\pi}(s',a')]
$$

$\bf{\underline{ Second \hspace{0.1cm} question :}} $

$$
Q^{*}(s,a) = \max_{\pi}E_{p^\pi}[\sum_{t\leq T} \gamma^{t} r(s_t,a_t)|s_0 = s, a_0 = a]\\
= \max_{\pi'} ( r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{\pi'}(s',a')) \\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)\max_{\pi'}Q^{\pi'}(s',a') \\
= r(s,a) + \gamma \sum_{s'}\sum_{a'}p(s',a'|s_0,a_0)Q^{*}(s',a') \\
= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
$$

$\bf{\underline{ Third \hspace{0.1cm} question :}}$
We can recognize the mean square error 
$$
\mathcal{L}(\theta)= E[(\hat{\theta} - \theta)^{2}]\\
E_{s' \sim \pi^*(.|s,a)}[\Vert Q^{*}(s,a,\theta^{*})-Q(s,a,\theta)\Vert^{2}\Vert^{2}]\\
E_{s' \sim \pi^*(.|s,a)}[\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}]
$$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [41]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):# we want to stores moves in a replay buffer
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]
        

    def random_access(self):
        indice = np.random.randint(0,len(self.memory))
        return self.memory[indice]

***
The pipeline we will use for training is given below:

In [42]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
        
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [43]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s): #ecrase la fonction learned_act de l'agent pour la remplacer par celle ci
        s = np.expand_dims(s,axis=0)
        a = np.argmax(self.model.predict(s))
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        #input states and target q are going to be used when we'll do the loss
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size): #on fait une batch gradient descent 
            ######## FILL IN
            state,next_state,action,reward, game_over = self.memory.random_access()
            input_states[i,:,:,:] = state  #pas besoin de redimensionner car on a précisé plus haut que inputs states est de dim 4
            state = np.expand_dims(state,axis = 0)
            target_q[i,:] = self.model.predict(state) #comme on veut faire la diff dans le MSE que pour l'action qu'on choisit, on met les autres valeurs égales comme ca différences entre Q et Q* = 0
            next_state = np.expand_dims(next_state, axis = 0)
            #ici on imite Q value function, et comme pour chaque i on a le même state, on pourra faire descente de gradient
            if game_over_: 
                target_q[i,action] = reward
            else:
                target_q[i,action] = reward + self.discount*np.max(self.model.predict(next_state))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q) #inputs state c'est les training inputs et target_q c'est target data
        #donc input_state on va le passer à notre réseaux de neurons

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state))) #n_state = 2 , et le rat voit 25 cases
        model.add(Dense(25, activation='relu'))
        model.add(Dense(4,activation = 'linear')) #4 car pour chaque state on aura 4 Q(s,a) pour les 4 actions différentes
        
        ####### FILL IN
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model
        model.summary()
       


In [44]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 20, prefix='fc_train')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 104       
Total params: 1,379
Trainable params: 1,379
Non-trainable params: 0
_________________________________________________________________
Epoch 000/020 | Loss 1.2597 | Win/lose count 5.5/2.0 (3.5)
Epoch 001/020 | Loss 1.4315 | Win/lose count 5.5/7.0 (-1.5)
Epoch 002/020 | Loss 2.0218 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/020 | Loss 1.9746 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/020 | Loss 2.0014 | Win/lose count 6.0/7.0 (-1.0)
Epoch 005/020 | Loss 1.8846 | Win/lose count 4.5/1.0 (3.5)
Epoch 006/020 | Loss 2.0036 | Win/lose count 4

In [45]:
HTML(display_videos('fc_train10.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [46]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, (3, 3),input_shape=(5,5, self.n_state), activation='relu')) #8 filters, kernel size = (3,3)
        model.add(Conv2D(32, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model
        


In [47]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,20,prefix='cnn_train')


Epoch 000/020 | Loss 1.7641 | Win/lose count 5.0/2.0 (3.0)
Epoch 001/020 | Loss 1.8294 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/020 | Loss 1.5766 | Win/lose count 2.5/6.0 (-3.5)
Epoch 003/020 | Loss 1.5312 | Win/lose count 9.5/12.0 (-2.5)
Epoch 004/020 | Loss 1.6716 | Win/lose count 11.5/4.0 (7.5)
Epoch 005/020 | Loss 1.8783 | Win/lose count 6.5/1.0 (5.5)
Epoch 006/020 | Loss 1.9296 | Win/lose count 6.5/3.0 (3.5)
Epoch 007/020 | Loss 1.8747 | Win/lose count 7.5/2.0 (5.5)
Epoch 008/020 | Loss 1.8695 | Win/lose count 3.0/2.0 (1.0)
Epoch 009/020 | Loss 1.8004 | Win/lose count 8.5/3.0 (5.5)
Epoch 010/020 | Loss 1.8726 | Win/lose count 8.0/1.0 (7.0)
Epoch 011/020 | Loss 1.9899 | Win/lose count 5.0/1.0 (4.0)
Epoch 012/020 | Loss 1.8842 | Win/lose count 8.0/3.0 (5.0)
Epoch 013/020 | Loss 1.8988 | Win/lose count 7.0/1.0 (6.0)
Epoch 014/020 | Loss 1.8816 | Win/lose count 8.0/1.0 (7.0)
Epoch 015/020 | Loss 1.8299 | Win/lose count 14.0/2.0 (12.0)
Epoch 016/020 | Loss 1.8843 | Win/lose count 8.5/1

In [48]:
HTML(display_videos('cnn_train10.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [49]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 104       
Total params: 1,379
Trainable params: 1,379
Non-trainable params: 0
_________________________________________________________________
Test of the CNN
Win/lose count 15.5/0. Average score (31.0)
Final score: 1.55
Win/lose count 23.5/0. Average score (39.0)
Final score: 3.9
Win/lose count 31.5/2.0. Average score (45.666666666666664)
Final score: 6.85
Win/lose count 21.0/0. Average score (44.75)
Final score: 8.95
Win/lose count 21.0/2.0. Average score (43.4)
Final score: 10.85
Win/lose count 20.5/2.0. Average score (42.333333333333336)

In [20]:
HTML(display_videos('cnn_test10.mp4'))

In [21]:
HTML(display_videos('fc_test10.mp4'))

I tried two types of optimizer : SGD (with and without momentum) and Adam. I chose to use Adam since the network learn better with this optimizer.
-Both algorithms must be improved since the agent is often stuck in a certain area. Indeed, the exploration is not effective and the agent is using to often the exploration for direct reward instead of exploring to have better long term reward. We can notice that convolution model seems to have better results than fully connected model. The intuition behind that is : in convolutional layer, each neuron is connected to a few nearby neurons in the previous layer and the same set of weights is used for every neuron. So we take advantage of the spatiality of the grid by connected just neurons by there neighbors. 
-The higher the temperature will be, the more reward there will be on the grid. And as they are more rewards, the average score for both model would grow up. For example, if we take a temperature of 0.3, we have a total result for CNN : 10.3 and for FC : 5.6. But if we take a temperature of 0.9, we have a toral result for CNN : 40.6 and for FC : 26.9

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [110]:
def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0
    discount_coef = 0.9
    agent.set_epsilon(0.9)
    print("Epsilon :",agent.epsilon)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        t = 0
        if e%5 == 0 and e != 0 :#decrease the epsilon
            agent.set_epsilon(agent.epsilon*discount_coef)
            print("Epsilon :",agent.epsilon)
        while not game_over:
            # The agent performs an action
            
            
            action = agent.act(state)
            

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
        
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    


In [111]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #c'est visited postion 

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #reward = self.board[self.x, self.y]
        
        game_over = self.t > self.max_time
        
        
        #reward = 0
        if train: #we are encouraging exploration in the following lines by adding a malus when we already have visited this state
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 1 
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        #state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
        #                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        #on prend aléatoirement de 3 à grid - 3 et [0] pour dire qu'on veut pas un array mais un réel
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0] 
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
    
        return state
    

    
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1
#
#reward = reward + self.board[self.x, self.y]
## 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [112]:
# Training
epochs_train = 30
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epsilon : 0.9
Epoch 000/030 | Loss 2.0043 | Win/lose count 17.0/12.0 (5.0)
Epoch 001/030 | Loss 2.1504 | Win/lose count 9.5/16.0 (-6.5)
Epoch 002/030 | Loss 1.9488 | Win/lose count 20.5/17.0 (3.5)
Epoch 003/030 | Loss 2.0146 | Win/lose count 15.5/20.0 (-4.5)
Epoch 004/030 | Loss 1.8628 | Win/lose count 17.5/21.0 (-3.5)
Epsilon : 0.81
Epoch 005/030 | Loss 2.1814 | Win/lose count 16.5/7.0 (9.5)
Epoch 006/030 | Loss 2.0448 | Win/lose count 18.5/12.0 (6.5)
Epoch 007/030 | Loss 2.0706 | Win/lose count 23.5/23.0 (0.5)
Epoch 008/030 | Loss 2.1367 | Win/lose count 11.0/20.0 (-9.0)
Epoch 009/030 | Loss 1.9574 | Win/lose count 15.0/15.0 (0.0)
Epsilon : 0.7290000000000001
Epoch 010/030 | Loss 2.0952 | Win/lose count 17.0/20.0 (-3.0)
Epoch 011/030 | Loss 1.9737 | Win/lose count 16.0/16.0 (0.0)
Epoch 012/030 | Loss 1.9247 | Win/lose count 14.5/17.0 (-2.5)
Epoch 013/030 | Loss 1.9465 | Win/lose count 16.0/9.0 (7.0)
Epoch 014/030 | Loss 2.0244 | Win/lose count 15.0/13.0 (2.0)
Epsilon : 0.656100000000

In [113]:
# Evaluation
epochs_test = 30
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 24.0/23.0. Average score (2.0)
Final score: 0.06666666666666667
Win/lose count 18.5/21.0. Average score (-1.5)
Final score: -0.1
Win/lose count 25.5/22.0. Average score (1.3333333333333333)
Final score: 0.13333333333333333
Win/lose count 21.5/15.0. Average score (4.25)
Final score: 0.5666666666666667
Win/lose count 20.0/27.0. Average score (0.6)
Final score: 0.1
Win/lose count 21.5/17.0. Average score (2.0)
Final score: 0.4
Win/lose count 12.0/10.0. Average score (2.2857142857142856)
Final score: 0.5333333333333333
Win/lose count 17.0/17.0. Average score (2.0)
Final score: 0.5333333333333333
Win/lose count 10.5/7.0. Average score (2.5555555555555554)
Final score: 0.7666666666666667
Win/lose count 14.5/16.0. Average score (2.0)
Final score: 0.6666666666666666
Win/lose count 16.0/9.0. Average score (3.090909090909091)
Final score: 1.1333333333333333
Win/lose count 18.0/16.0. Average score (3.1666666666666665)
Final score: 1.2666666666666666
Win/lose count 12.5/13.0. Averag

We can now see in the video that our agent is more able to visit unknown area as we add a penalty when we visit a state. 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [125]:
class MIM(Agent):
    def __init__(self, new_expert,grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(MIM, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        self.expert = new_expert #this is our expert agent that our actor will mims its behavior

    def learned_act(self, s): #ecrase la fonction learned_act de l'agent pour la remplacer par celle ci
        s = np.expand_dims(s,axis=0)
        a = np.argmax(self.expert.predict(s)) #we are now using the expert agent to mimic its actions
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size): #on fait une batch gradient descent 
            ######## FILL IN
            state,next_state,action,reward, game_over = self.memory.random_access()
            input_states[i,:,:,:] = state  #pas besoin de redimensionner car on a précisé plus haut que inputs states est de dim 4
            state = np.expand_dims(state,axis = 0)
            target_q[i,:] = self.model.predict(state) #comme on veut faire la diff dans le MSE que pour l'action qu'on choisit, on met les autres valeurs égales comme ca différences entre Q et Q* = 0
            next_state = np.expand_dims(next_state, axis = 0)
            #ici on imite Q value function, et comme pour chaque i on a le même state, on pourra faire descente de gradient
            if game_over_: 
                target_q[i,action] = reward
            else:
                target_q[i,action] = reward + self.discount*np.max(self.model.predict(next_state))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q) #inputs state c'est les training inputs et target_q c'est target data
        #donc input_state on va le passer à notre réseaux de neurons

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        


In [126]:
class DQN_CNN_MIM(MIM):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_MIM, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, (3, 3),input_shape=(5,5, self.n_state), activation='relu')) #8 filters, kernel size = (3,3)
        model.add(Conv2D(32, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model
        


In [127]:
epochs_train = 30
env_mim = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_mim = DQN_CNN_MIM(agent.model,size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_mim, env_mim, epochs_train, prefix='cnn_train_explore_mim')
HTML(display_videos('cnn_train_explore_mim10.mp4'))

Epsilon : 0.9
Epoch 000/030 | Loss 1.7295 | Win/lose count 15.0/20.0 (-5.0)
Epoch 001/030 | Loss 2.1172 | Win/lose count 10.5/9.0 (1.5)
Epoch 002/030 | Loss 1.7681 | Win/lose count 10.5/14.0 (-3.5)
Epoch 003/030 | Loss 1.5084 | Win/lose count 11.5/13.0 (-1.5)
Epoch 004/030 | Loss 1.7526 | Win/lose count 19.0/15.0 (4.0)
Epsilon : 0.81
Epoch 005/030 | Loss 1.5490 | Win/lose count 12.0/15.0 (-3.0)
Epoch 006/030 | Loss 1.7021 | Win/lose count 11.0/7.0 (4.0)
Epoch 007/030 | Loss 1.6602 | Win/lose count 13.0/16.0 (-3.0)
Epoch 008/030 | Loss 1.7184 | Win/lose count 8.5/12.0 (-3.5)
Epoch 009/030 | Loss 2.0131 | Win/lose count 11.5/13.0 (-1.5)
Epsilon : 0.7290000000000001
Epoch 010/030 | Loss 1.9322 | Win/lose count 5.5/8.0 (-2.5)
Epoch 011/030 | Loss 1.6869 | Win/lose count 5.0/10.0 (-5.0)
Epoch 012/030 | Loss 1.6791 | Win/lose count 8.5/7.0 (1.5)
Epoch 013/030 | Loss 1.8942 | Win/lose count 9.0/11.0 (-2.0)
Epoch 014/030 | Loss 2.1964 | Win/lose count 9.0/7.0 (2.0)
Epsilon : 0.6561000000000001

In [128]:
epochs_test = 30
test(agent_mim, env_mim,epochs_test,prefix='cnn_test_explore_mim')
HTML(display_videos('cnn_test_explore_mim10.mp4'))

Win/lose count 7.5/16.0. Average score (-17.0)
Final score: -0.5666666666666667
Win/lose count 9.5/6.0. Average score (-5.0)
Final score: -0.3333333333333333
Win/lose count 4.0/9.0. Average score (-6.666666666666667)
Final score: -0.6666666666666666
Win/lose count 7.0/9.0. Average score (-6.0)
Final score: -0.8
Win/lose count 7.0/7.0. Average score (-4.8)
Final score: -0.8
Win/lose count 6.0/5.0. Average score (-3.6666666666666665)
Final score: -0.7333333333333333
Win/lose count 4.0/7.0. Average score (-4.0)
Final score: -0.9333333333333333
Win/lose count 7.0/7.0. Average score (-3.5)
Final score: -0.9333333333333333
Win/lose count 3.5/8.0. Average score (-4.111111111111111)
Final score: -1.2333333333333334
Win/lose count 8.0/3.0. Average score (-2.7)
Final score: -0.9
Win/lose count 3.0/9.0. Average score (-3.5454545454545454)
Final score: -1.3
Win/lose count 6.0/7.0. Average score (-3.4166666666666665)
Final score: -1.3666666666666667
Win/lose count 3.0/4.0. Average score (-3.3076923

The Mimic DQN isn't outperforming the reference teaching agent but still do better than the agent without exploration. Nevertheless, Mimic method's is very useful when we are training a single deep policy network over a set of related source task. Indeed, a network trained using Actor-Mimic is capable of reaching expert performance on many games simultaneously. Moreover, using Actor-Mimic as a multitask pretraining phase can significantly improve learning speed.


***